## Comprehensive Code Set

In [1]:
def categorical_frequency_count(datasource,table_name,column_name,filter_list, threshold = 0.1,previous_results = {}):
        try:
            engine = create_engine(db_connection_string)
            filter_condition = f"cast({column_name} as varchar) in ({', '.join([f'\'{str(item)}\'' for item in filter_list])})"
            query = f"""
            select {column_name},count(*) as frequency from 
            {datasource}.{table_name}
            where {filter_condition}
            group by {column_name} 
            """
            with engine.connect() as conn:
                result = conn.execute(text(query)).fetchall()
            current_counts = {row[0]:row[1] for row in result}
            # print(current_counts)
            today = datetime.now().strftime('%m%d%Y')

            if today in previous_results:
                previous_counts = previous_results[today]
            else:
                if previous_results:
                    latest_date = max(previous_results.keys())
                   
                    previous_counts = previous_results[latest_date]
                
                else:
                    previous_counts = current_counts.copy()
              

            error_flag = 0
            error_flag_list = []
            for category,current_count in current_counts.items():
                previous_count = previous_counts.get(category,0)
                if previous_count>0:
                    percentage_change = abs((current_count - previous_count)/previous_count)
                else:
                    percentage_change = current_count
                if percentage_change>threshold:
                    error_flag = 1
                    error_flag_list.append(1)
                else:
                    error_flag_list.append(0)
                      
            previous_results[today] = current_counts
            return current_counts,previous_counts,error_flag,error_flag_list
                
        except Exception as e:
            print(e)

In [2]:
def dict_sum(combined_dict):
    print(combined_dict)
    result = {}
    if len(combined_dict) < 1:
        return {}
    for key,value in combined_dict.items():
        cons_value = {}
        for sub_dict in value:
            for sub_key,sub_value in sub_dict.items():
                cons_value[sub_key] = cons_value.get(sub_key,0) + sub_value
        result[key] = cons_value
    return result
    

In [ ]:
def combine_dictionaries(data,stringval):
    prev_count = []
    for i in data:
        if stringval in i:
           prev_count.append(i[stringval])
    return prev_count

In [ ]:
def fetch_values(keys,data_dict):
    result = {}
    for key in keys:
        
        if key in data_dict:
            result[key] = data_dict[key]
        
    return result

In [ ]:
from datetime import datetime
def comprehensive_codeset_count(measure,final_list):
    measure_to_value_filtered = measure_to_value[measure_to_value['Measure ID']==measure]
    value_sets_possible_list = measure_to_value_filtered['Value Set Name'].unique()

    #filter relevant value sets in value to code table
    value_to_code_filtered = value_to_code[value_to_code['Value Set Name'].isin(value_sets_possible_list)] 
    code_system_possible_list = value_to_code_filtered['Code System'].unique()

    code_system_dictionary = {}
    code_system_list_current = []
    code_system_list_previous = []
    code_system_dict_current = {}
    code_system_dict_previous = {}
    for code_system in code_system_possible_list:


        # print("---------------------------------------------")
        print(code_system)
        code_to_table_filtered = code_to_table[code_to_table['Code System']==code_system]
        value_set_code_system_filtered = value_to_code_filtered[value_to_code_filtered['Code System'] == code_system]
        codes  = value_set_code_system_filtered['Code'].unique()
        final_result_list = []
        for index,row in code_to_table_filtered.iterrows():
            code_system = row['Code System']
            datasource = row['Dat Source']
            table_name = row['Table Name']
            column_name = row['Column Name']
            # print(code_system,datasource,table_name,column_name)
            # for value in value_sets_possible_list:
            filtered_data = [d for d in final_list if d.get('Measure')==measure]
           
            # print("Filtered_Data")
            # print(filtered_data)

            result = None
            for d in filtered_data:
                if code_system in d['Previous Count']:
                    result = d
                
                
            if result is not None:  
                previous_results = {}
                date_prev = result['DVE Run Date']
            
                previous_results[date_prev] = result['Previous Count'][code_system]
                # print("Inside of if loop")
                # print(previous_results)
            else:
                previous_results = {}
                    
                        
            if datasource == 'HXG':
                datasource = 'dbo'
            current_counts,previous_results,error_flag,error_flag_list = categorical_frequency_count(datasource,table_name,column_name,codes,previous_results = previous_results)
            # print("Previous Results")
            # print(previous_results)
            # print("Current Counts")
            # print(current_counts)
            
            if code_system not in code_system_dict_current:
                code_system_dict_current[code_system] = [current_counts]
            else:
                code_system_dict_current[code_system].append(current_counts)

            # print("Previous Results")
            # print(previous_results)
            if result is None:   
                if code_system not in code_system_dict_previous:
                    code_system_dict_previous[code_system] = [previous_results]
                else:
                    code_system_dict_previous[code_system].append(previous_results)
            else:
                if code_system not in code_system_dict_previous:
                    code_system_dict_previous[code_system] = [previous_results]
                else:
                    code_system_dict_previous[code_system] = [previous_results]

    combined_dict_current = dict_sum(code_system_dict_current)
    combined_dict_previous = dict_sum(code_system_dict_previous)
    # combined_dict_currentcs[code_system] = combined_dict_current
    # combined_dict_previouscs[code_system] = combined_dict_previous

    print("Current Result")
    print(combined_dict_current)

    print("Previous Result")
    print(combined_dict_previous)

    value_set_final_list = []
    for value in value_sets_possible_list:
        print(value)
        value_set_final_dict = {}
        individual_value_set = value_to_code_filtered[value_to_code_filtered['Value Set Name']==value]
        code_systems_individual_value_sets = individual_value_set['Code System'].unique()
        values_current = fetch_values(code_systems_individual_value_sets,combined_dict_current) 

        values_previous = fetch_values(code_systems_individual_value_sets,combined_dict_previous)

        value_set_final_dict['DVE Run Date'] = datetime.now().strftime('%m%d%Y')
        value_set_final_dict['Measure'] = measure
        value_set_final_dict['Value Set'] = value
        value_set_final_dict['Current Count'] = values_current
        value_set_final_dict['Previous Count'] = values_previous
        value_set_final_dict['Error Flag Individual'] = error_flag_list
        value_set_final_dict['Error Flag'] = error_flag
        value_set_final_list.append(value_set_final_dict)
    return value_set_final_list
        


## Hospice